Tratamento geral dos dados
==========================

Esse `notebook` tem por objetivo realizar o primeiro contato com os dados que serão, posteriormente, utilizados para induzir `modelos` `preditivos`. De forma geral, o que será realizado é a remoção de `dados` `faltantes`; a remoção de linhas (exemplos) `redundantes`, ou seja, que não agregam `informação` ao problema e podem comprometre a métrica de desempenho do modelo; a conversão de dados `categóricos` (apenas uma coluna se encaixa nesse exemplo) em `numéricos`; e a separção em `features` e `targets`. Ao final, será gerado um conjunto de `Dataframes` que será utilizado para induzir os `modelos` `preditivos`. Esse dataframe será do tipo _ComPode começarma-Separated Values_ (CSV).

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

## Importando o `Dataframe`

Podemos carregar o `Dataframe`, obtido por meio da referência [1], no programa por meio da `biblioteca` `pandas`. O arquivo é um CSV. Após ler o arquivo, podemos mostrar parte do `Dataframe` para criar a referência tabular dos dados.

Junto à importação, podemos retirar exemplos que tenham, por ventura, dados faltantes.

In [2]:
# Os índices já são definidos no próprio arquivo com a coluna "rownames"
df = pd.read_csv('Dataset_Alzheimers.csv', index_col='rownames', sep=',' )
df.dropna(axis=0, inplace=True)
df

,ACE_CD143_Angiotensin_Converti,ACTH_Adrenocorticotropic_Hormon,AXL,Adiponectin,Alpha_1_Antichymotrypsin,Alpha_1_Antitrypsin,Alpha_1_Microglobulin,Alpha_2_Macroglobulin,Angiopoietin_2_ANG_2,Angiotensinogen,...,VEGF,Vitronectin,von_Willebrand_Factor,age,tau,p_tau,Ab_42,male,Genotype,Class
rownames,,,,,,,,,,,,,,,,,,,,,
1,2.003100,-1.386294,1.098387,-5.360193,1.740466,-12.631361,-2.577022,-72.650290,1.064711,2.510547,...,22.034564,-0.040822,-3.146555,0.987624,6.297754,4.348108,12.019678,0,E3E3,Control
2,1.561856,-1.386294,0.683282,-5.020686,1.458615,-11.909882,-3.244194,-154.612278,0.741937,2.457283,...,18.601843,-0.385662,-3.863233,0.986150,6.659294,4.859967,11.015759,0,E3E4,Control
3,1.520660,-1.714798,-0.145276,-5.809143,1.193922,-13.642963,-2.882404,-136.529178,0.832909,1.976365,...,17.476191,-0.223144,-3.540459,0.986667,6.270988,4.400247,12.302271,1,E3E4,Control
4,1.680826,-1.609438,0.683282,-5.115996,1.280934,-15.523564,-3.170086,-98.361752,0.916291,2.376085,...,17.545595,-0.653926,-3.863233,0.986702,6.152733,4.494886,12.398138,0,E3E4,Control
5,2.400931,-0.967584,0.190890,-4.779524,2.128232,-11.133063,-2.343407,-144.944601,0.955511,2.862219,...,20.778602,0.166216,-3.816713,0.987163,6.623707,4.524589,11.024109,0,E3E3,Control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,1.719055,-1.660731,0.828427,-5.449140,1.098612,-16.321511,-3.324236,-160.010404,0.955511,1.908566,...,17.517898,-0.446287,-3.729701,0.983687,6.132899,4.430662,11.913409,0,E3E3,Control
330,1.391905,-1.514128,-0.145276,-4.906275,1.609438,-11.838035,-2.120264,-154.612278,0.405465,1.789161,...,15.618049,0.205439,-4.509860,0.984351,4.805741,3.357803,12.878940,1,E3E4,Control
331,1.001198,-1.347074,-0.010025,-4.509860,1.193922,-14.406260,-3.170086,-179.087488,-0.248461,2.596327,...,14.543388,-0.478036,-4.509860,0.986877,5.551835,3.768614,9.508593,0,E3E3,Control


## Definindo as features e targets

Para continuar, devemos diferenciar as `colunas` que fornecem `informação` acerca do caso e as que definem o que queremos descobrir sobre o indivíduo. Nesse caso, o objetivo é prever se há, ou não, predisposição à alzheimer, baseando a análise em todos os outros dados fronecidos. Assim, nosso `target` será `categórico` (_Control ou Impaired_) e nossas `features` devem ser `numéricas` para o funcionamento dos `algoritmos`.

OBS: As `variáveis` `features` e `targets` são `listas` com _nomes_ de `colunas`.

In [3]:
# As features são todas as colunas com exceção da coluna "Class"
features = df.columns.tolist()
features.remove('Class')

# O target é a coluna "Class"
target = ['Class']

## Eliminando dados redundantes

Dados, ou seja, exemplos, que sejam iguais ou muito próximos podem caracterizar `vazamento` de `dados`. Isso significa que quando o `desempenho` do modelo for medido, o valor pode estar sendo superestimando, em rasão de ele ter sido treinado com dados "já conhecidos". Uma estratégia para evitar isso é agrupar esses exemplos e colapssá-los em um único valor, sua `moda`. Primeiramente, pode-se `arredondar` os valores para evitar superestimar a precisão do equipamento, depois executa-se o procedimento de `colapso`.

OBS: A `moda` é usada nesse caso pois o `target` que será `previsto` é uma _classe_, não um número.

In [4]:
casas_arredondamento = 5 # Define quantas casas decimais o número vai ter

df_round = df.round(casas_arredondamento)

# Agrupa os targets por meio das features. Caso existam dois exemplos iguais, atribui àquela combinação de valores, a moda dos targets agrupados
grupo = df_round.groupby(features, sort=False)[target].agg(pd.Series.mode)
df_tratado = grupo.reset_index()

C:\Users\joao25006\AppData\Local\Temp\ipykernel_20056\2210588839.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_tratado = grupo.reset_index()
C:\Users\joao25006\AppData\Local\Temp\ipykernel_20056\2210588839.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_tratado = grupo.reset_index()
C:\Users\joao25006\AppData\Local\Temp\ipykernel_20056\2210588839.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining

## Conversão simbólico-numérica

Como é possível perceber, há, entre as `features`, um dado que é categórico binário. Para torná-lo numérico é possível utilizar a função `.get_dummies` para realizar a conversão do tipo `OneHot`. A conversão só precisa ser realizada nas features então, na `célula` abaixo o `Dataframe` de `features` é separado. Por facilidade para os próximo passos, o Dataframe de teste também é separado.

In [5]:
df_features = df_tratado[features]
df_target = df_tratado[target]

In [6]:
df_features = pd.get_dummies(df_features, dtype=int, drop_first=True)
features = df_features.columns.tolist()


### Organizando

Para definir bem o que já foi feito até agora, a `célula` abaixo reúne as `variáveis` _importantes_ (que serão utilizadas nos próximos passos) criadas até o momento. Uma `lista` com as _features_; um `Dataframe` com as _features_; uma `lista` ocm o _target_; um `Dataframe` com o _target_

OBS: O ponto e vírugula ao final serve para que o `Jupyter` não exiba o `Dataframe` e gera poluição visual no `notebook`.

In [7]:
# Features
features
df_features

# Target
target
df_target;

## Split em treino e teste (`holdout`)

É de grande importância, após treinar um modelo preditivo, testar a "eficiência" desse moodelo em prever o que foi pedido. Uma estratégia para avaliar o desempenho de um modelo é, após seu treinamento, realizar uma "prova" com dados que o _modelo nunca tenha tido contato de nenhuma forma_. Isso permite avaliar a _taxa de acerto_ do modelo. Para `regressores`, essa medida está, muitas vezes, relacionada a um erro (valor) na previsão; para `classificadores`, essa medida tende a estar ligada a um tipo de `acurácia`, no sentido da _razão acertos e tentativas_.

O split em treino e teste pode ser realizado com facilidade por meio da função `train_test_split` da biblioteca `sklearn`

Para começar devemos definir quantos dados serão utilizados para `teste` e quantos ficaram para `treino`. Percentualmente isso tende a variar entre _10 e 30%_. Dado que após todo o processamento ainda existem 333 exemplos, é razoável aplicar 25% do `dataset` para _teste_, dado que ainda assim exixtirão 249 _exemplos_ para _treinar_ o modelo.

In [8]:
holdout = 0.25
seed = 1141

Note que os `Dataframes` de `features` e `target` foram criados anteriormente.

In [9]:
indices = df_tratado.index

# faz o Holdout
indices_treino, indices_teste = train_test_split(
    indices, test_size=holdout, random_state=seed, stratify=df_target
)

# Separa em variáveis de treino e variáveis de teste
# Features
FEATURES_TREINO = df_features.loc[indices_treino]
FEATURES_TESTE = df_features.loc[indices_teste]

# Target
TARGET_TREINO = df_target.loc[indices_treino]
TARGET_TESTE = df_target.loc[indices_teste]

print(f'O número TOTAL de dados é {len(indices)}')
print(f'O número de dados para treino é {len(FEATURES_TREINO)}')
print(f'O número de dados para teste é de {len(FEATURES_TESTE)}')

O número TOTAL de dados é 333
O número de dados para treino é 249
O número de dados para teste é de 84


## Exportando os dados

Os `modelos` que serão treinados terão seu `algoritmos` em _arquivo diferente_, mas precisam ter acesso aos dados tratados neste `notebook`. Para isso, os `Dataframes` gerados após o pré-processamento serão `exportados` como _arquivos CSV_ para que possam ser `importados` nos outros `notebooks`

In [10]:
FEATURES_TREINO.to_csv('Features para treino.csv')
FEATURES_TESTE.to_csv('Features para teste.csv')
TARGET_TREINO.to_csv('Target para treino.csv')
TARGET_TESTE.to_csv('Target para teste.csv')

## Referências

1. Artigo relacionado ao `Dataset`, disponívle em: 